In [4]:
import os
import json
from pathlib import Path
from langchain.text_splitter import TokenTextSplitter, CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma, Qdrant
from langchain.document_loaders import TextLoader

In [5]:
report = "A"
new_info_from_memory = "B"
prompt = (
            f"The following report was prepared:\n{report}\n"
            f"The following new information was found:\n{new_info_from_memory}\n"
            """The report must have the following json format and using proper indentations and json formatting: {{ Report: [{{ "{0}": "{1}", "{2}": "{3}", "{4}": [{5}] }}]}}""".format("Question", "questions description", "Answer", "answer description", "Sources", "list of web links to the sources") + "\n"
            f"""Update the report with the new information. For the questions: {report}"""
            f"Make the report as concise and brief as possible focusing only on the most important information and facts."
        )

In [6]:
print(prompt)

The following report was prepared:
A
The following new information was found:
B
The report must have the following json format and using proper indentations and json formatting: { Report: [{ "Question": "questions description", "Answer": "answer description", "Sources": [list of web links to the sources] }]}
Update the report with the new information. For the questions: AMake the report as concise and brief as possible focusing only on the most important information and facts.


In [7]:
keys_file = Path(".").resolve().parent / "keys.json"
print(keys_file)
with open(keys_file) as f:
    keys = json.load(f)
os.environ["OPENAI_API_KEY"] = keys["OPENAI_API_KEY"]

D:\00Repos\GPT-Swarm\keys.json


In [8]:
embeddings = OpenAIEmbeddings()

In [9]:
persist_directory = Path("D:\\00Repos\\GPT-Swarm\\tmp\\swarm\\shared_memory")
vectordb = Chroma(persist_directory=str(persist_directory), embedding_function=embeddings)
retriever_chroma = vectordb.as_retriever(search_type="mmr")

Using embedded DuckDB with persistence: data will be stored in: D:\00Repos\GPT-Swarm\tmp\swarm\shared_memory


In [10]:
vectordb._collection.count()

408

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chains.question_answering import load_qa_chain

In [12]:
model = ChatOpenAI(model='gpt-3.5-turbo', temperature=0) # 'ada' 'gpt-3.5-turbo' 'gpt-4',
qa_chain = load_qa_chain(model, chain_type="stuff")
qa = RetrievalQA(combine_documents_chain=qa_chain, retriever=retriever_chroma)

In [13]:
"""
    - Generate a comprehensive description of the startup. Describe the value proposition, the product, USP and business model of a startup.
    - Find any mentions of the startup in the news, social media, etc.
    - Find top companies and startups in this field. Find out their locations, raised funding, value proposition, differentiation, etc.
    - Find top investors in this field. Including specific details like 'company AAA (link) invested in company BBB (link) $XX in year YYYY'
    - Describe the market size, growth rate and trends of this field.
    - Main problems and challenges of the field. Create an extensive list of problems. What can stop the field from growing? What can stop the company from succeeding?
    - Briefly describe the technology for the non-tech audience. Include links to the main articles in the field.
    - Who are the main influencers in this field?
    - What questions should we ask the startup to make a more informed decision? Avoid generic and obvious questions and focus on field/domain specific questions that can uncover problems with this specific startup.
"""

context = (
    "A new startup just send us their pitch. Find if the startup is worth investing in. The startup is called Brainamics and it is in the space of brain computer interfaces."
)

questions = [
    "Generate a comprehensive description of the startup. Describe the value proposition, the product, USP and business model of a startup.",
    "Find any mentions of the startup in the news, social media, etc.",
    "Find top companies and startups in this field. Find out their locations, raised funding, value proposition, differentiation, etc.",
    "Find top investors in this field. Including specific details like 'company AAA (link) invested in company BBB (link) $XX in year YYYY'",
    "Describe the market size, growth rate and trends of this field.",
    "Main problems and challenges of the field. Create an extensive list of problems. What can stop the field from growing? What can stop the company from succeeding?",
    "Briefly describe the technology for the non-tech audience. Include links to the main articles in the field.",
    "Who are the main influencers in this field?",
    "What questions should we ask the startup to make a more informed decision? Avoid generic and obvious questions and focus on field/domain specific questions that can uncover problems with this specific startup."
]

for question in questions:  
    result = qa.run(context+question+"Be very brief and concise. Focus on the essential information and provide https links to the sources.")
    print(f"=====================================================================================================")
    print(f"=====================================================================================================")
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result} \n")

-> **Question**: Generate a comprehensive description of the startup. Describe the value proposition, the product, USP and business model of a startup. 

**Answer**: Brainamics is a startup operating in the brain computer interface industry, aiming to enhance human performance through its technology. Their value proposition is to develop brain computer interfaces that can be integrated into various devices to improve performance in gaming, education, and healthcare. The product uses electroencephalography (EEG) technology to measure brain activity and translate it into commands that control devices. The unique selling point of Brainamics is that their brain computer interfaces are affordable, user-friendly, and compatible with a wide range of devices, making them accessible and scalable. The business model generates revenue through a combination of software licensing fees and consulting services, providing a diversified income stream. Sources: 
1. https://brainamics.co/
2. https://www.